In [71]:
import tensorflow as tf # a default graph is created with a tf import
import numpy as np

In [45]:
# the contents of these variables are the *outputs* of the operations
# and not operations themselves

a = tf.constant(5, name = "my_a") # output of a const value
b = tf.constant(2, name = "my_b")
c = tf.constant(3, name = "my_c")

d = tf.multiply(a, b, name = "my_d")
e = tf.add(c, b, name = "my_e")
f = tf.subtract(d, e, name = "my_f")
g = tf.logical_xor(True, False)

### Expanding dims

In [101]:
A = tf.constant([[1,2,3], [4,5,6]])
x = tf.constant([7, 8, 9])
print(A.get_shape()) 
print(x.get_shape())
# need to reshape x
x = tf.expand_dims(x, 1)
print(x.get_shape())
b = tf.matmul(A, x)

sess = tf.InteractiveSession()
print(b.eval())
sess.close()

(2, 3)
(3,)
(3, 1)
[[ 50]
 [122]]


### Fetches

In [46]:
# the graph computes only essential nodes according to the 
# dependencies (here - of "outs")
with tf.Session() as sess:
    fetches = [a,b,c,d,e,f]  # fetches hold the element of the graph
                             # we want to compute
    outs = sess.run(fetches) # execution
    
print(outs)
print(type(outs[0]))         # list of numpy elements

[5, 2, 3, 10, 5, 5]
<type 'numpy.int32'>


In [47]:
# we can perform the same runs in the same graph multiple times
# but each in a new session
with tf.Session() as sess:
    out = sess.run(f)
print(type(f))
print(f)
print(type(out))
print(out)

<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("my_f:0", shape=(), dtype=int32)
<type 'numpy.int32'>
5


### Graphs

In [12]:
# compare the default graph and another one
print(tf.get_default_graph())
g = tf.Graph()
print(g)

# check which graph is associated with a node
s = tf.constant(10)
print(s.graph)

# use contect manager (with statement) to run with a given graph
with g.as_default():
    print(g is tf.get_default_graph())

True


In [49]:
with tf.Graph().as_default():
    print(tf.constant(1, name = "my")) # names should be unique *in the same graph*;
    print(tf.constant(2, name = "my")) # tf adds "_<n>" otherwise

Tensor("my:0", shape=(), dtype=int32)
Tensor("my_1:0", shape=(), dtype=int32)


### Name scopes

In [52]:
with tf.Graph().as_default():
    with tf.name_scope("prefix_A"):
        print(tf.constant(1, name = "p")) 
        print(tf.constant(2, name = "q"))
        with tf.name_scope("prefix_B"):
            print(tf.constant(11, name = "r"))
            print(tf.constant(22, name = "s"))

Tensor("prefix_A/p:0", shape=(), dtype=int32)
Tensor("prefix_A/q:0", shape=(), dtype=int32)
Tensor("prefix_A/prefix_B/r:0", shape=(), dtype=int32)
Tensor("prefix_A/prefix_B/s:0", shape=(), dtype=int32)


### Variables

Variables are a special type of Tensors that can maintain a "fixed" state in a graph.
Using them requires they are initialized.

In [65]:
init_val = tf.random_normal((1, 4), 0 , 1)
var = tf.Variable(init_val)
print(var)

try:
    with tf.Session() as sess:
        result = sess.run(var)
except tf.errors.FailedPreconditionError:
    print "It'll throw with a mesg Attempting to use uninitialized value Variable_6"

# Now initialize the Variable
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    result = sess.run(var)
    print(result)
    
# with each run a new Variable is created - the name will have "_n" appended to name: 'Variable_11:0'

<tf.Variable 'Variable_11:0' shape=(1, 4) dtype=float32_ref>
It'll throw with a mesg Attempting to use uninitialized value Variable_6
[[ 2.53858614  0.71087223  0.24701412  0.54690635]]


### Placeholders

In [69]:
# if shape is None or empty, anu size is accepted
pa = tf.placeholder(tf.float32, shape=(None, 10))
print(pa)

Tensor("Placeholder_1:0", shape=(?, 10), dtype=float32)


In [86]:
_x = np.random.randn(5, 10)
_y = np.random.randn(10, 1)

with tf.Graph().as_default():
    x = tf.placeholder(tf.float32, shape=(5, 10))
    y = tf.placeholder(tf.float32, shape=(10, 1))
    xy = tf.matmul(x, y)
    argmax = tf.argmax(xy)

    with tf.Session() as sess:
        matr = sess.run(xy, feed_dict={x: _x, y: _y})
        # scalar = sess.run(argmax, feed_dict={x: _x, y: _y}) # hm, why does this work??
        scalar = sess.run(argmax, feed_dict={xy: matr})

print(matr)
print(scalar)

[[ 2.61419582]
 [-1.43373728]
 [ 1.55462527]
 [ 0.58991057]
 [ 1.69534183]]
[0]


### Optimization

#### Multivariate linear regression

In [90]:
x = tf.placeholder(tf.float32, shape=(None, 3))
y_true = tf.placeholder(tf.float32, shape=(None, 3))
w = tf.Variable([[0,0,0]], dtype=tf.float32, name = 'weights')
b = tf.Variable(0, dtype=tf.float32, name = 'intercept')

y_pred = tf.matmul(tf.transpose(w), x) + b

#### Loss function

In [93]:
# Mean Square Error
loss_mse = tf.reduce_mean(tf.square(y_true - y_pred))

# Cross entropy (esp. for categorical data)
loss_cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred))

#### Optimize the loss function

In [99]:
# In Tensorflow gradients are calculated using "automatic differentiation"
# learning rate - determines how large is the step in the direction of negative gradient
# (we don't want to overshoot the minimum)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss_mse)